In [2]:
import pandas as pd

In [45]:
# labeled images from test_20
test_20_labeled = pd.read_csv('../_data/df_csv/test20_labeled.csv', index_col=[0])

# labeled images from test_20 without complex
test_20_labeled_without_complex = pd.read_csv('../_data/df_csv/test_20_labeled_without_complex.csv', index_col=[0])

# labeled images from test_20 where only one label
test_20_only_one_label = pd.read_csv('../_data/df_csv/test_20_only_one_label.csv', index_col=[0])

# original labels
orig_labels = pd.read_csv('../_data/df_csv/train_0.csv')

# duplicates 21
duplicates = pd.read_csv('../_data/df_csv/list_to_drop.csv')

# labels without duplicates 21
labels = pd.read_csv('../_data/df_csv/train_labels.csv')

# labels 20
labels_2020 = pd.read_csv('../_data/df_csv/train_20.csv')

# bounding boxes model from colab 999 images
df_crop1 = pd.read_csv('../_data/df_csv/df_boxes.csv' , index_col=[0]) #2021
df_crop2 = pd.read_csv('../_data/df_csv/df_boxes_20.csv', index_col=[0]) #2020
df_crop = pd.read_csv('../_data/df_csv/df_boxes_colab.csv', index_col=[0]) #20+21

# labels 21+20 without comlex20
labels_21_20 = pd.read_csv('../_data/df_csv/labels_21_20.csv', index_col=[0])

# labels 21+20 without comlex20 plus wrongs upsampled and augmented
labels_21_20_w = pd.read_csv('/app/_data/df_csv/labels_21_20_plus_wrong_upsample.csv')

# labels 21+20 without comlex20 plus labeled images from test_20 with complex
labels_21_20_plus_test20 = pd.read_csv('../_data/df_csv/labels_21_20_plus_test20.csv', index_col=[0])



# labels 21+20 without comlex20 plus labeled images from test_20 without complex
labels_21_20_plus_test20_without_complex = pd.read_csv('../_data/df_csv/labels_21_20_plus_test20_without_complex.csv', index_col=[0])

# predictions wrong >=8 models effnet
wrong_eff4_ns = pd.read_csv('/app/sandbox/wrong_predictions/eff4/wrong_eff4_ns.csv',  index_col=[0])
wrong_eff4_ids = pd.read_csv('/app/sandbox/wrong_predictions/eff4/wrong_eff4_ids.csv',  index_col=[0])

In [39]:
test_20_labeled_without_complex

,image,labels,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
0,Test_1592.jpg,scab,0,0,0,0,1
1,Test_1089.jpg,scab,0,0,0,0,1
2,Test_1636.jpg,rust,0,0,0,1,0
3,Test_1639.jpg,scab,0,0,0,0,1
4,Test_1641.jpg,scab,0,0,0,0,1
...,...,...,...,...,...,...,...
888,Test_66.jpg,scab,0,0,0,0,1
889,Test_70.jpg,scab,0,0,0,0,1
890,Test_72.jpg,scab,0,0,0,0,1
891,Test_73.jpg,scab,0,0,0,0,1


In [40]:
test_20_labeled

,image,labels,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
0,Test_1592.jpg,scab,0,0,0,0,0,1
1,Test_1089.jpg,scab complex,1,0,0,0,0,1
2,Test_1636.jpg,rust,0,0,0,0,1,0
3,Test_1639.jpg,scab,0,0,0,0,0,1
4,Test_1641.jpg,scab,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
888,Test_66.jpg,scab,0,0,0,0,0,1
889,Test_70.jpg,scab,0,0,0,0,0,1
890,Test_72.jpg,scab,0,0,0,0,0,1
891,Test_73.jpg,scab,0,0,0,0,0,1


In [41]:
test_20_only_one_label.to_csv('../_data/df_csv/test_20_only_one_label.csv')

In [42]:
import os
import json
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

%matplotlib inline

import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from PIL import Image

from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit,
    train_test_split,
)
from tensorflow import keras
from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB7, EfficientNetB0
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import notebook, tqdm

In [43]:
PATH = "/app/_data/"
PATH_JSON1 = '/app/_data/json/1071_big.json'
PATH_JSON2 = '/app/_data/json/part_2_bb.json'
BATCH_SIZE = 64
SEED = 37
IMAGE_SIZE = 224
NUM_CLASSES = 6
TRAIN_IMG_PATH = "/app/_data/train_images/"
TEST_IMG_PATH = "/app/_data/test_images/"
feature_columns = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']

In [44]:
# labels = labels.join(labels["labels"].str.get_dummies(sep=" "))
# feature_columns = labels.columns[2:].tolist()
# labels_2020["image_id"] = labels_2020["image_id"] + ".jpg"
# labels_2020.columns = ["image", "healthy", "complex", "rust", "scab"]
# labels_2020 = labels_2020.query("complex !=1").reset_index(drop=True)
# labels_2020 = labels_2020[["image", "healthy", "rust", "scab"]]
# feature_columns_20 = ["healthy", "rust", "scab"]
# for i in labels_2020.index:
#     labels_2020.loc[i, "labels_20"] = "".join(
#         list(
#             map(
#                 lambda x, y: x * y,
#                 labels_2020.loc[i, feature_columns_20].values,
#                 feature_columns_20,
#             )
#         )
#     )
# labels_21_20 = pd.concat([labels, labels_2020], axis=0, join="outer", ignore_index=True)
# labels_21_20[feature_columns] = labels_21_20[feature_columns].fillna(0).astype("int")
# labels_21_20["labels"] = labels_21_20["labels"].fillna(labels_21_20["labels_20"])
# labels_21_20 = labels_21_20.drop("labels_20", axis=1)
# # labels_21_20.to_csv('../_data/df_csv/labels_21_20.csv')
# labels_21_20.tail(2)

In [7]:
# labels_21_20_plus_test20 =  pd.concat([labels_21_20, test_20_labeled], axis=0, join="outer", ignore_index=True)

In [8]:
# labels_21_20_plus_test20.to_csv('../_data/df_csv/labels_21_20_plus_test20.csv')


In [46]:
def count_unique_labels(df):
    col = feature_columns
    dct = {}
    for i in col:
        cnt = df[df[i]==1].shape[0]
        dct[i]=cnt
    return dct

In [47]:
d = count_unique_labels(labels_21_20_plus_test20_without_complex)

In [48]:
d

{'complex': 2120,
 'frog_eye_leaf_spot': 4428,
 'healthy': 5172,
 'powdery_mildew': 1275,
 'rust': 2774,
 'scab': 7038}

In [49]:
def get_imgs_upsample(df, num_samples, num_frac):
    """
    df - dataframe
    num_samples - до скольки увеличивать топ6 labels
    num_frac - во сколько раз увеличивать топ6-12 labels
    """
    val_count = df['labels'].value_counts()
    top_6 = val_count.index[:6].tolist()
    top_6_12 = val_count.index[6:12]
    data_top_6 = pd.DataFrame()
    data_top_6_12 = pd.DataFrame()
    for i in top_6:
        cnt = val_count[i]
        if cnt<=num_samples:
            sample = df[df['labels']==i].sample(n=num_samples-cnt, replace=True, random_state=SEED)
            data_top_6 = pd.concat([data_top_6, sample], axis=0, ignore_index=True)
    for i in top_6_12:
        cnt = val_count[i]
        if 500<cnt<=1000:
            sample = df[df['labels']==i].sample(n=num_samples-cnt, replace=True, random_state=SEED)
            data_top_6_12 = pd.concat([data_top_6_12, sample], axis=0, ignore_index=True)
        else:
            sample = df[df['labels']==i].sample(n=cnt*num_frac, replace=True, random_state=SEED)
            data_top_6_12 = pd.concat([data_top_6_12, sample], axis=0, ignore_index=True)
    final_df = pd.concat([data_top_6, data_top_6_12], axis=0, ignore_index=True)
    return final_df

In [50]:
def get_imgs_upsample(df, num_samples, num_frac):
    """
    df - dataframe
    num_samples - до скольки увеличивать топ6 labels
    num_frac - во сколько раз увеличивать топ6-12 labels
    """
    val_count = df['labels'].value_counts()
    top_12 = val_count.index[:12].tolist()
    data = pd.DataFrame()
    for i in top_12:
        cnt = val_count[i]
        if 666<cnt<=num_samples:
            sample = df[df['labels']==i].sample(n=num_samples-cnt, replace=True, random_state=SEED)
            data = pd.concat([data, sample], axis=0, ignore_index=True)
        if cnt<=666:
            sample = df[df['labels']==i].sample(n=cnt*num_frac, replace=True, random_state=SEED)
            data = pd.concat([data, sample], axis=0, ignore_index=True)
    final_df = data
    return final_df

In [55]:
ups_df = get_imgs_upsample(df = labels_21_20_w, num_samples=2000, num_frac=3)

In [56]:
ups_df['labels'].value_counts()

frog_eye_leaf_spot complex         1980
rust frog_eye_leaf_spot            1416
scab frog_eye_leaf_spot complex    1200
rust complex                       1092
powdery_mildew complex             1044
powdery_mildew                      745
Name: labels, dtype: int64

In [57]:
labels_ups = pd.concat([labels_21_20_w, ups_df], axis=0, ignore_index=True)
labels_ups['labels'].value_counts()
labels_21_20_w['labels'].value_counts()

scab                               5737
healthy                            5234
frog_eye_leaf_spot                 3286
complex                            2861
scab frog_eye_leaf_spot            2728
frog_eye_leaf_spot complex         2640
rust                               2593
powdery_mildew                     2000
scab frog_eye_leaf_spot complex    2000
rust frog_eye_leaf_spot            1888
rust complex                       1456
powdery_mildew complex             1392
Name: labels, dtype: int64

scab                               5737
healthy                            5234
frog_eye_leaf_spot                 3286
complex                            2861
scab frog_eye_leaf_spot            2728
rust                               2593
powdery_mildew                     1255
scab frog_eye_leaf_spot complex     800
frog_eye_leaf_spot complex          660
rust frog_eye_leaf_spot             472
rust complex                        364
powdery_mildew complex              348
Name: labels, dtype: int64

In [59]:
labels_ups = labels_ups.drop('Unnamed: 0', axis=1)

In [66]:
labels_ups = labels_ups.sample(frac=1).reset_index(drop=True)

In [68]:
labels_ups.to_csv('/app/_data/df_csv/labels_21_20_plus_wrong_upsample_plus_upsample.csv')